## Calculate ERPs

In [ ]:
#Importing files and modules

import numpy as np
import matplotlib.pyplot as plt
from os import path
%matplotlib notebook
from brainpipe.system import study
from brainpipe.visual import *
from brainpipe.statistics import *
from mne.baseline import rescale
from mne.filter import filter_data
import pandas as pd
from pandas import ExcelWriter

from detect_peaks import detect_peaks

## User variables

In [ ]:
low_pass_filter = 10.
sf = 512.
norm_mode = 'mean' #'ratio' 'mean' 'percent' 
baseline = [717 , 768]
data_to_use = [768, 1536]
n_perm = 200

## ERP Amplitude - Latency - 1 elec

In [ ]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/TS_E_all_cond_by_block_trigs_th40_art400_30_250/')
path_to_save = path.join (st.path, 'feature/ERP_Encoding_all_bipo_100ms_mean_thr40_art400_30_250/By_trial_max/')

elec = 23
su = 'VACJ'

filename = su+'_E1E2_concat_allfilter1_bipo.npz'
data_all = np.load(path.join(path_data, filename))
data, channel, label = data_all['x'], data_all['channel'], data_all['label']

# Select data for one elec + name :
data_elec = data[elec,:,:]
ntrials = len(data_elec[2])
#print ('\nOriginal data : ', data.shape, 'Channel : ', channel[elec], 'Label : ', label[elec], 'N_trials :', ntrials, 'One elec shape : ', data_elec.shape)

#Filter data for one elec (all trials):
data = np.array(data_elec, dtype='float64')
data_to_filter = np.swapaxes(data, 0, 1)
filtered_data = filter_data(data_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
filtered_data = np.swapaxes(filtered_data, 0, 1)
#print ('Size of filtered data:', filtered_data.shape,)

#Normalize the non-averaged data (all trials)
times = np.arange(filtered_data.shape[0])
#print ('time points : ', times.shape)
filtered_data_to_norm = np.swapaxes(filtered_data, 0, 1)
norm_filtered_data = rescale(filtered_data_to_norm, times=times, baseline=baseline, mode=norm_mode)
norm_filtered_data = np.swapaxes(norm_filtered_data, 0, 1)
#print ('Size norm & filtered data : ', norm_filtered_data.shape,)

# =======================COMPUTE AMPLITUDE AND LATENCY FOR EACH TRIAL=====================================
# Select data
concat_min = []
concat_max = []
for trial in range(0, ntrials, 1):
    norm_filtered_data_to_plot = norm_filtered_data[range(baseline[0], data_to_use[1])]
    time_x = 1000 * np.arange(baseline[0]-baseline[1], norm_filtered_data_to_plot.shape[0]+baseline[0]-baseline[1],) / sf
    data_trial = norm_filtered_data[range(baseline[0], data_to_use[1]), trial]
    print ('size of norm and fitlered data to use : ', data_trial.shape, 'time size : ', time_x.shape,)

    #Minimums local
    indexes_min = detect_peaks(data_trial, mph=25, valley = True, mpd=100,edge='both', show=False,)
    min_trial_ampl = np.array(data_trial[indexes_min])
    min_trial_time = np.array(time_x[indexes_min])
    min_trial = np.stack([min_trial_ampl, min_trial_time], axis=0)
    concat_min.append(min_trial)
    min_array = np.concatenate(concat_min, axis=-1)
    min_array = np.transpose(min_array, axes=None)
    #print ('min array', min_array)
    
    #Maximums local
    indexes_max = detect_peaks(data_trial, mph=25, mpd=100,edge='both', show=False,)
    max_trial_ampl = np.array(data_trial[indexes_max])
    max_trial_time = np.array(time_x[indexes_max])
    max_trial = np.stack([max_trial_ampl, max_trial_time], axis=0)
    concat_max.append(max_trial)
    max_array = np.concatenate(concat_max, axis=-1)
    max_array = np.transpose(max_array, axes=None)
    #print ('max array', max_array)
    
    #plot the figure
    fig = plt.figure(0, figsize=(12, 7))
    ax = fig.add_subplot(111)
    fig.subplots_adjust(top=0.85)
    ax.set_xlabel('Times (ms)', fontsize=12)
    ax.set_ylabel('Potential', fontsize=12)
    plt.plot(time_x, data_trial, '#808080', linewidth=1.5, label='data')
    #pplot(times_plot, data_trial_to_plot, indexes)
    lines = [0]
    addLines(plt.gca(), vLines=lines, vColor=['firebrick']*2, vWidth=[2]*2, hLines=[0], hColor=['#000000'], hWidth=[2])
    plt.plot(min_trial_time, min_trial_ampl, 'b+', markersize=8, markeredgewidth=1.5 )
    plt.plot(max_trial_time, max_trial_ampl, 'r+', markersize=8, markeredgewidth=1.5)
    plt.grid()
    #plt.show()

    #save the figures
    fname = (path_to_save+ su+'_E1E2_concat_all_'+str(label[elec])+'_trial_num_'+str(trial)+'.png')
    print (fname)
    plt.savefig(fname, dpi=300, bbox_inches='tight')
    plt.close()

# Save max min 
df_min = pd.DataFrame(min_array, columns = ('Min_Peak_Ampl', 'Min_Peak_Timing'))
df_max = pd.DataFrame(max_array, columns = ('Max_Peak_Ampl', 'Max_Peak_Timing'))
df_tot = df_max.append(df_min)
print (df_tot)
writer = ExcelWriter(path_to_save+su+'_min_max_peak_'+str(label[elec])+'.xlsx')
df_tot.to_excel(writer)
writer.save()